In [8]:
### clean data

import pandas as pd
import sys
from pathlib import Path
data = pd.read_parquet("../data/raw/v2025.12.08.1716/broadband_processed_data.parquet").drop_duplicates()

In [10]:
data_cleaned = data.drop(columns=["description", 'payment_method', 'transaction_type'])
cleaned = data.loc[:, ['primary_merchant', 'transaction_classification_0', 'transaction_classification_1', 'customer_id', 'account_id', 'date', 'amount', 'credit_debit']]
cleaned = cleaned.loc[cleaned['primary_merchant'] != '']
classification_stats = cleaned.groupby('transaction_classification_0')['amount'].agg(
        avg_amount_by_classification='mean',
        total_txn_by_classification='count'
    )
cleaned = cleaned.merge(classification_stats, on='transaction_classification_0', how='left')
cleaned
cleaned.to_parquet("../data/cleaned/columns_selected.parquet", index=False)

In [24]:
# Merchants with empty/missing classifications
cleaned[
    (cleaned['transaction_classification_0'].isna() | (cleaned['transaction_classification_0'] == '')) |
    (cleaned['transaction_classification_1'].isna() | (cleaned['transaction_classification_1'] == ''))
]['primary_merchant'].value_counts()

primary_merchant
PRET    178
Name: count, dtype: int64

In [ ]:
# Classification name standardization mapping
# Consolidates duplicates and compound classifications

classification_mapping = {
    "": "Uncategorized",
    "Bills and Utilities": "Bills & Utilities",
    "Pension and insurances": "Pension and Insurances",
    "Travel & Transport": "Travel",
    "Auto & Transport|Food & Dining": "Auto & Transport",
    "Entertainment|Food & Dining": "Entertainment",
    "Entertainment|Personal Services": "Entertainment",
    "Food & Dining|Home": "Food & Dining",
    "Food & Dining|Personal Services": "Food & Dining",
    "Food & Dining|Shopping": "Food & Dining",
    "Personal Services|Shopping": "Personal Services"
}

# Apply mapping to the cleaned data
cleaned['transaction_classification_0'] = cleaned['transaction_classification_0'].replace(classification_mapping)

# Show impact
print(f"Classifications after standardization: {cleaned['transaction_classification_0'].nunique()}")
print("\nValue counts:")
print(cleaned['transaction_classification_0'].value_counts())

In [ ]:
# Save standardized data
cleaned.to_parquet("../data/cleaned/columns_selected.parquet", index=False)
print("Saved standardized data to ../data/cleaned/columns_selected.parquet")